In [1]:
#!pip install pillow datasets
from datasets import load_dataset
import os
from PIL import Image


# Load the ImageNet-1k training, validation, and test datasets
train_dataset = load_dataset("imagenet-1k", split="train")
val_dataset = load_dataset("imagenet-1k", split="validation")
test_dataset = load_dataset("imagenet-1k", split="test")


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Loading dataset shards:   0%|          | 0/257 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/25 [00:00<?, ?it/s]

In [2]:


def save_imagenet_format(dataset, save_path):
    """Save a Hugging Face dataset in ImageNet folder format"""
    os.makedirs(save_path, exist_ok=True)
    
    skipped_count = 0
    for i in range(len(dataset)):
        try:
            # Access each sample directly by index to avoid the iterator
            sample = dataset[i]
            
            class_id = str(sample["label"]) 
            class_folder = os.path.join(save_path, class_id)
            os.makedirs(class_folder, exist_ok=True)
            
            # Save image
            img = sample["image"]
            img_path = os.path.join(class_folder, f"{i}.jpg")
            img.save(img_path)
            
            if i % 100000 == 0:
                print(f"Saved {i} images... (Skipped: {skipped_count})")
                
        except Exception as e:
            skipped_count += 1
            print(f"Skipping image {i} due to error: {e}")
            continue
    
    print(f"Processing complete. Total images skipped: {skipped_count}")


# Save the ImageNet train dataset in ImageNet folder format
#save_imagenet_format(train_dataset, "datasets/imagenet_full_size/061417/train")

# Save the ImageNet validation dataset in ImageNet folder format
save_imagenet_format(val_dataset, "datasets/imagenet_full_size/061417/val")

# Save the ImageNet test dataset in ImageNet folder format
save_imagenet_format(test_dataset, "datasets/imagenet_full_size/061417/test")



Saved 0 images... (Skipped: 0)
Processing complete. Total images skipped: 0
Saved 0 images... (Skipped: 0)


/root/miniconda3/envs/myenv/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:949: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


Processing complete. Total images skipped: 0


In [4]:
import os

def count_images(directory):
    image_count = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff')):
                image_count += 1
    return image_count

directory = "data/datasets/imagenet_full_size/061417/val"
image_count = count_images(directory)
print(f"Total images in {directory}: {image_count}")

directory = "data/datasets/imagenet_full_size/061417/test"
image_count = count_images(directory)
print(f"Total images in {directory}: {image_count}")

Total images in data/datasets/imagenet_full_size/061417/val: 0
Total images in data/datasets/imagenet_full_size/061417/test: 0


In [1]:
import os
import random

# Define the path to your ImageNet dataset
dataset_dir = 'ijepa/src/datasets/imagenet_full_size/061417/train'

# Define the path where the text file will be saved
output_txt = 'ijepa/src/datasets/imagenet_1_percent.txt'


# Open the text file to write
with open(output_txt, 'w') as f:
    # Loop over each class folder (0 to 999)
    for class_id in range(1000):
        class_folder = os.path.join(dataset_dir, str(class_id))
        
        # Check if the folder exists
        if os.path.exists(class_folder):
            # Get all images in the class folder
            image_files = [img for img in os.listdir(class_folder) if img.endswith(('jpg', 'jpeg', 'png'))]
            
            # Determine the 1% sample size
            total_images = len(image_files)
            sample_size = int(total_images*(1/100))

            # Sample images
            sampled_images = random.sample(image_files, sample_size)
            
            # Write the image file paths and class labels to the text file
            for image in sampled_images:
                img_name = image
                f.write(f'{class_id}_{img_name}\n')
print("Balanced 1% dataset created and saved as imagenet_1_percent_balanced.txt")




Balanced 1% dataset created and saved as imagenet_1_percent_balanced.txt


In [15]:
!pip install torchvision 
from ijepa.src.datasets.imagenet1k import make_imagenet1k
make_imagenet1k(
        transform=transform,
        batch_size=batch_size,
        collator=mask_collator,
        pin_mem=pin_mem,
        training=True,
        num_workers=num_workers,
        world_size=world_size,
        rank=rank,
        root_path=root_path,
        image_folder=image_folder,
        copy_data=True,
        drop_last=True,
        subset_file="src/datasets/imagenet_1_percent.txt"  # should point to your 1% subset
    )


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 240.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 252.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 262.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 260.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 622.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 204.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 206.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 214.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 216.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 217.3 MB/

NameError: name 'transform' is not defined